# Crawl Data from Top Youtuber in Taiwan

##  1. Get Famous Youtuber Link

In [1]:
'''
###########################################################
    Import Libraries
###########################################################
'''
import numpy as np
import requests
import sys
import math
import time
from bs4 import BeautifulSoup
import functools

### Get the top 250 youtubers link and basic attributes in taiwan. Sorted by subscribers num

In [39]:
'''
######################################################################################################
    Get the top 250 youtubers link in socialblade website
######################################################################################################
'''

top_youtuber_rank_url = "https://socialblade.com/youtube/top/country/tw/mostsubscribed"

content = requests.get(top_youtuber_rank_url).text
page_soup = BeautifulSoup(content, "html.parser")

# Find potential user position 
div_class = page_soup.find_all('a', href=True)

youtuber_socialblade = [] # Name, url in socialblade
for num in range(0, len(div_class)):
    href = div_class[num]
    url = href.get('href')
    if url.find("/youtube/user/") is not -1:
        youtuber_socialblade.append([href.text, "https://socialblade.com"+url])

print("Top 250 Youtuber: ")
for i in range(0, len(youtuber_socialblade)):
    print(i+1, 'is', youtuber_socialblade[i][0], youtuber_socialblade[i][1])

Top 250 Youtuber: 
1 is 這群人TGOP https://socialblade.com/youtube/user/e84768131
2 is 滾石唱片 ROCK RECORDS https://socialblade.com/youtube/user/rockrecordstaipei
3 is 相信音樂BinMusic https://socialblade.com/youtube/user/binmusictaipei
4 is 阿滴英文 https://socialblade.com/youtube/user/eo3jwe3hezuwfdvcehqk9q
5 is 蔡阿嘎 https://socialblade.com/youtube/user/kyoko38
6 is 阿神 https://socialblade.com/youtube/user/charlie615119
7 is 杰威爾音樂 JVR Music https://socialblade.com/youtube/user/jvrmuzic
8 is 華納音樂 Warner Music Taiwan https://socialblade.com/youtube/user/warnertaiwan
9 is 聖結石Saint https://socialblade.com/youtube/user/idhd_1spj49unbwx1fjs2a
10 is 福茂唱片 https://socialblade.com/youtube/user/linfairrecords
11 is 華研國際 https://socialblade.com/youtube/user/himservice
12 is 眾量級CROWD https://socialblade.com/youtube/user/3lbfxbwtebdooub8-qjm9q
13 is 放火 Louis https://socialblade.com/youtube/user/louislee0602
14 is 瑪莎與熊 Masha and The Bear CH https://socialblade.com/youtube/user/fsxflbi8fpnavq_rl3i4bw
15 is 小玉 https

In [184]:
'''
######################################################################################################
    Get youtuber name, url, UPLOADS, SUBSCRIBERS, VIDEO VIEWS, Country, CHANNEL TYPE, USER CREATED DATE
######################################################################################################
'''
youtuber = []
for i in range(0, len(youtuber_socialblade)): 
    user = youtuber_socialblade[i]
    url = user[1]
    content = requests.get(url).text
    page_soup = BeautifulSoup(content, "html.parser")
    attributes = []
    attributes.append(user[0])
    
    # Get youtube url
    youtube_link_class = page_soup.find_all("a", {"class": "core-button -margin core-small-wide ui-black"})
    for link_text in youtube_link_class:
        link_url = link_text.get('href')
        if link_url is not None:
            attributes.append(link_url)
    
    # Get attributes value
    info_class = page_soup.find_all("div", {"class": "YouTubeUserTopInfo"})
    for info in info_class:
        value = info.find_all("span", {"style": "font-weight: bold;"})[0].text
        attributes.append(value)
    youtuber.append(attributes)
    time.sleep(0.05)
print("Done")

Done


In [166]:
print(type(youtuber))
for i in range(0, len(youtuber)):
    for j in youtuber[i]:
        print(j, end=" "),
    print()

<class 'list'>
這群人TGOP https://youtube.com/channel/UC6FcYHEm7SO1jpu5TKjNXEA 175 2,775,729 588,570,794 TW Entertainment Jun 7th, 2008 
滾石唱片 ROCK RECORDS https://youtube.com/channel/UCPcF3KTqhD67ADkukx_OeDg 4,170 2,215,651 2,504,752,950 TW Music Jun 8th, 2010 
相信音樂BinMusic https://youtube.com/channel/UC7ovu6a8ydIbDy0fAKmoZ9A 1,192 2,109,314 2,279,440,922 TW Music Aug 10th, 2009 
阿滴英文 https://youtube.com/channel/UCeo3JwE3HezUWFdVcehQk9Q 345 1,917,965 138,965,633 TW Education Jan 11th, 2015 
蔡阿嘎 https://youtube.com/channel/UCPwxSX0DYDMQxCvgfeVDv_g 543 1,915,788 418,102,547 TW Entertainment Jun 3rd, 2006 
阿神 https://youtube.com/channel/UCnJEWsS5agXCkqIpyHC9Grg 3,488 1,714,232 638,694,014 TW Games May 11th, 2009 
杰威爾音樂 JVR Music https://youtube.com/channel/UCKUlsqazP-4QmxdEtUPlxOA 516 1,443,235 2,002,918,978 TW Music Sep 2nd, 2010 
華納音樂 Warner Music Taiwan https://youtube.com/channel/UCStpPtoeQUYII1Jsbl0qf8A 1,860 1,413,156 944,611,583 TW Music Jul 6th, 2009 
聖結石Saint https://youtube.com/cha

In [188]:
'''
######################################################################################################
    Preprocess data about comma and date
######################################################################################################
'''
month_dictionary = {'Jan':'01', 'Feb':'02', "Mar":'03', "Apr":'04', "May":'05', "Jun":'06', 
                    "Jul":'07', "Aug":'08', "Sep":'09', "Oct":'10', "Nov":'11', "Dec":'12'}
date_dictionary = {'1':'01', '2':'02', '3':'03', '4':'04', '5':'05', '6':'06', '7':'07', '8':'08', '9':'09'}

def remove_comma(value_string):
    if type(value_string) == int:
        return value_string
    value = 0
    for i in range(0, len(value_string)):
        if(value_string[i]!=","):
            value = value*10 + int(value_string[i])
    return value
       
def date_to_int(value_string):
    if type(value_string) == int:
        return value_string
    value_string = value_string.replace(',', " ")
    value_string = value_string.replace('th', "")
    value_string = value_string.replace('rd', "")
    value_string = value_string.replace('nd', "")
    value_string = value_string.replace('st', "")
    
    date = value_string.split()
    date[0] = month_dictionary[date[0]]
    if(len(date[1]) == 1):
        date[1] = date_dictionary[date[1]]
    
    date = 
    print (date)
    
    return value_string
    
for i in range(0, len(youtuber_copy)):
    youtuber[i][2] = remove_comma(youtuber[i][2]) #UPLOADS
    youtuber[i][3] = remove_comma(youtuber[i][3]) #SUBSCRIBERS
    youtuber[i][4] = remove_comma(youtuber[i][4]) #VIDEO VIEWS
    youtuber[i][7] = date_to_int(youtuber[i][7]) #Date from string to int
    
    
for i in range(0, len(youtuber)):
    for j in youtuber[i]:
        print(j, end=" "),
    print()

['06', '07', '2008']
['06', '08', '2010']
['08', '10', '2009']
['01', '11', '2015']
['06', '03', '2006']
['05', '11', '2009']
['09', '02', '2010']
['07', '06', '2009']
['06', '10', '2013']
['02', '24', '2009']
這群人TGOP https://youtube.com/channel/UC6FcYHEm7SO1jpu5TKjNXEA 175 2775729 588570794 TW Entertainment Jun 7  2008 
滾石唱片 ROCK RECORDS https://youtube.com/channel/UCPcF3KTqhD67ADkukx_OeDg 4170 2215651 2504752950 TW Music Jun 8  2010 
相信音樂BinMusic https://youtube.com/channel/UC7ovu6a8ydIbDy0fAKmoZ9A 1192 2109314 2279440922 TW Music Aug 10  2009 
阿滴英文 https://youtube.com/channel/UCeo3JwE3HezUWFdVcehQk9Q 345 1917965 138965633 TW Education Jan 11  2015 
蔡阿嘎 https://youtube.com/channel/UCPwxSX0DYDMQxCvgfeVDv_g 543 1915788 418102547 TW Entertainment Jun 3  2006 
阿神 https://youtube.com/channel/UCnJEWsS5agXCkqIpyHC9Grg 3488 1714232 638694014 TW Games May 11  2009 
杰威爾音樂 JVR Music https://youtube.com/channel/UCKUlsqazP-4QmxdEtUPlxOA 516 1443235 2002918978 TW Music Sep 2  2010 
華納音樂 Warner Mus